In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

np.random.seed(42)

X, y = make_blobs(n_samples=200, centers=2)
idxs_1 = np.argwhere(y==1)
idxs_0 = np.argwhere(y==0)
X1 = X[np.reshape(idxs_1, idxs_1.size)]
X0 = X[np.reshape(idxs_0, idxs_0.size)]

plt.plot(X0[:,0], X0[:,1], 'b.', X1[:,0], X1[:,1], 'r.')
y[y==0] -= 1

In [ ]:
class SVM:
    """ 
    A regularized kernel SVM 
    
    ...

    Attributes
    ----------
    C : float
        The inverse regularization parameter
    kernel: str
        The kernel type (e.g. linear, gaussian_rbf, etc..)

    Methods
    -------
    train(X,y):
        Trains the model on the dataset (X,y)

    predict(q):
        Makes a prediction about query point q
    """
    def __init__(self, C, kernel="linear"):
        self.C = C
        self.kernel = kernel
        self.r_idx = 0

    def _compute_w(self, X, y, alpha):
        """ Use representer theorem to retrieve w from alpha """
        N, D = X.shape

        w = np.zeros(D)
        for i in range(N):
            w += alpha[i]*y[i]*X[i]

        return w

    def _compute_b(self, y, w, alpha):
        pass
    
    def _float_equals(self, a, b, tol=1e-6):
        return abs(a - b) < tol

    def _has_converged(self, X, y, alpha, K, eps, j):
        """ Check if alpha values have converged """
        N, D = X.shape
        
        w = self._compute_w(X, y, alpha)
        b, found = self._compute_b(y, w, alpha)
        
        if not found:
            return False, w, b

        u = []
        for i in range(N):
            u_i = 0
            for j in range(N):
                u_i += y[j]*alpha[j]*K[j,i]
            
            u_i -= b
            u.append(u_i)
        
        converged = True
        for i in range(N):
            if alpha[i] == 0: #self._float_equals(alpha[i], 0):
                if not y[i]*u[i] >= 1: #1 - eps:
                    self.r_idx = i
                    converged = False
                    break
            elif alpha[i] == self.C: #self._float_equals(alpha[i], self.C):
                if not y[i]*u[i] <= 1: #1 - eps:
                    self.r_idx = i
                    converged = False
                    break
            else:
                if not self._float_equals((y[i]*u[i]).item(), 1, tol=eps):
                    self.r_idx = i
                    converged = False
                    break
                
        return converged, w, b
    
    def train(self, X, y):
        """ Train the classifier using the SMO algorithm """
        N, D = X.shape

        # Picking feasible alpha
        alpha = np.zeros(N)
        eps = 1e-3
        converged = False

        # Compute Kernel
        if self.kernel == "gaussian":
            K = self.compute_gaussian_kernel(X)
        else:
            K = self.compute_linear_kernel(X)

        i = 1 # use i (as idx of s) and i-1 (as idx of r) as alpha idxs
        num_iters = 0
        while not converged:
            pass

    def compute_gaussian_kernel(self, X, sigma=1.0):
        N, D = X.shape
        
        K = np.zeros((N,N))
        for i in range(N):
            for j in range(N):
                K[i,j] = np.exp(-(X[i]-X[j])@(X[i]-X[j])/(2*sigma**2))
                
        return K

    def compute_linear_kernel(self, X):
        return X@X.T

    def predict(self, q):
        return 1 if self.w@q + self.b >= 0 else -1

In [ ]:
C = 1

# def normalize(X):
#     return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# X_tilde = normalize(X)

svm = SVM(C, kernel="linear")
svm.train(X, y.astype('float64'))

In [ ]:
plt.plot(X0[:,0], X0[:,1], 'b.', X1[:,0], X1[:,1], 'r.')
svm.predict(np.array([-4, 6]))

prediction_grid = np.zeros((200, 200))

x = np.linspace(-6, 8, 200)
y = np.linspace(-1, 12, 200)

xv, yv = np.meshgrid(x, y, indexing='xy')
for i in range(200):
    for j in range(200):
        prediction_grid[i,j] = svm.predict(np.array([xv[j,i], yv[j,i]]))

plt.figure()
plt.imshow(prediction_grid)